<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/BILSTM%2BGLOVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
device_name = tf.test.gpu_device_name()
print(device_name)
# df_train = pd.read_csv('/content/SA-emotion-train_val_test_clean_tweets.csv')
df = pd.read_csv('/content/SA-emotion-train_val_test_clean_tweetsekphrasis.csv', sep='\t')
# df_train = df_train.sample(frac=1)
df = df.sample(frac=1)

In [ ]:
drive.mount('/content/drive')

In [ ]:
nltk.download('all')

In [ ]:
df.head()

In [ ]:
df.emotion.unique()

In [ ]:
category_dict = {0:[1,0,0,0,0],1:[0,1,0,0,0],2:[0,0,1,0,0],3:[0,0,0,1,0],4:[0,0,0,0,1]}
data_tweet = [x.lower().split() for x in df['tweets']]
data_cat = np.array([category_dict[x] for x in df['emotion']])
data_tweet = data_tweet

In [ ]:
X_train_data, X_test_data, y_train_data, y_test_data = train_test_split(data_tweet, data_cat, test_size=0.2, random_state=42)
X_train_data, X_val_data, y_train_data, y_val_data = train_test_split(X_train_data, y_train_data, test_size=0.2, random_state=42)

#Glove

In [ ]:
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [ ]:
batch_size = 500
Max_input_size = max([len(x) for x in data_tweet])
mul_factor=1.5

In [ ]:
positive = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/positive-words.csv')
positive = list(positive['0'][1:])
negative = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/negative-words.csv')
negative = list(negative['0'][2:])
pn_dict = {}
for i in range(len(positive)):
  pn_dict[positive[i]] = 1
for i in range(len(negative)):
  pn_dict[negative[i]] = -1

In [ ]:
X_train_norm = X_train_data
X_test_norm = X_test_data
X_val_norm = X_val_data
X_train_mult = [nltk.pos_tag(x) for x in X_train_data]
X_test_mult = [nltk.pos_tag(x) for x in X_test_data]
X_val_mult = [nltk.pos_tag(x) for x in X_val_data]
X_train_negpos = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_train_mult]
X_test_negpos = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_test_mult]
X_val_negpos = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_val_mult]

y_train = y_train_data
y_test = y_test_data
y_val = y_val_data


#Normal

In [ ]:
def glove_gen_norm(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size=100):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([vocab[i] for i in x if i in vocab.keys()]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [ ]:
epochs=27
Embedding_size=200
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

# validation_generator = glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size,mul_factor)
train_generator = glove_gen_norm(vocab_glove,X_train_norm,y_train,batch_size,Max_input_size,Embedding_size)
val_generator = glove_gen_norm(vocab_glove,X_val_norm,y_val,batch_size,Max_input_size,Embedding_size)
history_norm = model.fit(train_generator,validation_data=val_generator, validation_steps=100,steps_per_epoch=int(len(X_train_norm)/batch_size),epochs=epochs)
history_norm_eval = model.evaluate(x = glove_gen_norm(vocab_glove,X_test_norm,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))


In [ ]:
print(history_norm.history['loss'])
print(history_norm.history['accuracy'])
print(history_norm.history['val_accuracy'])


In [ ]:
plt.title("Normal: training_accuracy, validation_accuracy vs epochs")
plt.plot(history_norm.history['accuracy'],c='r',label='training_accuracy')
plt.plot(history_norm.history['val_accuracy'],c='b',label='validation_accuracy')
plt.xlabel('epochs')
plt.legend()
# plt.yticks()
plt.show()
print(history_norm_eval)

Mult

In [ ]:
def glove_gen_mult(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ' or tweet[i][1][:2]=='RB':
            tweet[i]=((tweet[i][0])*mul_factor,tweet[i][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [ ]:
epochs=27
Embedding_size=200
model_mult = tf.keras.Sequential()
model_mult.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_mult.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_mult.add(tf.keras.layers.Dropout(0.2))
model_mult.add(tf.keras.layers.Dense(64,activation='relu'))
model_mult.add(tf.keras.layers.Dropout(0.2))
model_mult.add(tf.keras.layers.Dense(5, activation='softmax'))
model_mult.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

# validation_generator = glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size,mul_factor)
train_generator = glove_gen_mult(vocab_glove,X_train_mult,y_train,batch_size,Max_input_size,Embedding_size,mul_factor)
val_generator = glove_gen_mult(vocab_glove,X_val_mult,y_val,batch_size,Max_input_size,Embedding_size,mul_factor)
history_mult = model_mult.fit(train_generator,validation_data=val_generator, validation_steps=100,steps_per_epoch=int(len(X_train_norm)/batch_size),epochs=epochs)
history_mult_eval = model_mult.evaluate(x = glove_gen_mult(vocab_glove,X_test_mult,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))


In [ ]:
plt.title("Mult: training_accuracy, validation_accuracy vs epochs")
plt.plot(history_mult.history['accuracy'],c='r',label='training_accuracy')
plt.plot(history_mult.history['val_accuracy'],c='b',label='validation_accuracy')
plt.xlabel('epochs')
plt.legend()
# plt.yticks()
plt.show()
print(history_mult_eval)

mult+negpos

In [ ]:
def glove_gen_negpos(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1],i[2]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          tweet[i]=((tweet[i][0]*tweet[i][2]),tweet[i][1],tweet[i][2])
        for i in range(l):
          if tweet[i][1][:2]=='JJ' or tweet[i][1][:2]=='RB':
            tweet[i]=((tweet[i][0]*mul_factor),tweet[i][1],tweet[i][2])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [ ]:
epochs=27
Embedding_size=200
model_negpos = tf.keras.Sequential()
model_negpos.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_negpos.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_negpos.add(tf.keras.layers.Dropout(0.2))
model_negpos.add(tf.keras.layers.Dense(64,activation='relu'))
model_negpos.add(tf.keras.layers.Dropout(0.2))
model_negpos.add(tf.keras.layers.Dense(5, activation='softmax'))
model_negpos.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

# validation_generator = glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size,mul_factor)
train_generator = glove_gen_negpos(vocab_glove,X_train_negpos,y_train,batch_size,Max_input_size,Embedding_size,mul_factor)
val_generator = glove_gen_negpos(vocab_glove,X_val_negpos,y_val,batch_size,Max_input_size,Embedding_size,mul_factor)
history_negpos = model_mult.fit(train_generator,validation_data=val_generator, validation_steps=100,steps_per_epoch=int(len(X_train_norm)/batch_size),epochs=epochs)
history_negpos_eval = model_mult.evaluate(x = glove_gen_negpos(vocab_glove,X_test_negpos,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))


In [ ]:
plt.title("Mult+negpos: training_accuracy, validation_accuracy vs epochs")
plt.plot(history_negpos.history['accuracy'],c='r',label='training_accuracy')
plt.plot(history_negpos.history['val_accuracy'],c='r',label='validation_accuracy')
plt.xlabel('epochs')
plt.legend()
# plt.yticks()
plt.show()
print(history_negpos_eval)

mult+negpos+addition

In [ ]:
def glove_gen_addi(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1],i[2]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          tweet[i]=((tweet[i][0]*tweet[i][2]),tweet[i][1],tweet[i][2])
        for i in range(l):
          if tweet[i][1][:2]=='JJ' or tweet[i][1][:2]=='RB':
            tweet[i]=((tweet[i][0]*mul_factor),tweet[i][1],tweet[i][2])
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='NN':
                tweet[z]=((tweet[z][0]+tweet[i][0]),tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]+tweet[i][0]),tweet[z][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [ ]:
epochs=27
Embedding_size=200
model_addi = tf.keras.Sequential()
model_addi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_addi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_addi.add(tf.keras.layers.Dropout(0.2))
model_addi.add(tf.keras.layers.Dense(64,activation='relu'))
model_addi.add(tf.keras.layers.Dropout(0.2))
model_addi.add(tf.keras.layers.Dense(5, activation='softmax'))
model_addi.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

# validation_generator = glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size,mul_factor)
train_generator = glove_gen_addi(vocab_glove,X_train_negpos,y_train,batch_size,Max_input_size,Embedding_size,mul_factor)
val_generator = glove_gen_addi(vocab_glove,X_val_negpos,y_val,batch_size,Max_input_size,Embedding_size,mul_factor)
history_negpos = model_addi.fit(train_generator,validation_data=val_generator, validation_steps=100,steps_per_epoch=int(len(X_train_norm)/batch_size),epochs=epochs)
history_negpos_eval = model_addi.evaluate(x = glove_gen_addi(vocab_glove,X_test_negpos,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))


In [ ]:
plt.title("Mult+negpos+addi: training_accuracy, validation_accuracy vs epochs")
plt.plot(history_negpos.history['accuracy'],c='r',label='training_accuracy')
plt.plot(history_negpos.history['val_accuracy'],c='b',label='validation_accuracy')
plt.xlabel('epochs')
plt.legend()
# plt.yticks()
plt.show()
print(history_negpos_eval)